![GCAT | Genomes for Life](images/logo-GCAT.png)

# Filter medication

In [1]:
import difflib
import re
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pylab import *

%matplotlib inline

## Load tables

- Qüestionari
- Participants

In [2]:
mica = pd.read_csv("/home/labs/dnalab/share/lims/R/gcat-cohort/output/export/QUESTIONARI/data.csv", low_memory=False)
participants = pd.read_csv("/home/labs/dnalab/share/lims/R/gcat-cohort/output/export/Participants/data.csv")

all = participants
all = pd.merge(all, mica, on=['entity_id'])

/software/debian-8/general/virtenvs/jupyter/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (22,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
coded = list(all.columns[all.columns.str.contains('_MEDICACION')])
columns = coded.copy()
columns.append('entity_id')

## Coded medication

Falten els AINES!!

In [4]:
others = list(all.columns[all.columns.str.contains('_MEDICACION_.*_TIPO')])
columns = others.copy()
columns.append('entity_id')

ds = all.loc[:, all.columns.isin(columns)]

In [5]:
coded_meds = pd.melt(ds, id_vars = ['entity_id'], value_vars = list(others))
coded_meds = coded_meds[~coded_meds.value.isnull()]
coded_meds.head()

,entity_id,variable,value
1051,=E00251510427221,CANCER1_MEDICACION_1_TIPO,H03AA;EUTIROX
1129,=E00251430309221,CANCER1_MEDICACION_1_TIPO,L02BA;TAMOXIFENO
1918,=E00251430207121,CANCER1_MEDICACION_1_TIPO,L02BA;TAMOXIFENO
1934,=E00251430252321,CANCER1_MEDICACION_1_TIPO,L02BA;TAMOXIFENO
2111,=E00251510807521,CANCER1_MEDICACION_1_TIPO,L01DC;MITOMYCIN C


In [6]:
def to_illness(colname):
    colname = re.sub(r"(_MEDICACION)", "", colname)
    colname = re.sub(r"(^MEDICACION_)", "", colname)
    colname = re.sub(r"(_TIPO)", "", colname)
    colname = re.sub(r"(_OTROS)", "", colname)
    colname = re.sub(r"(_\d)", "", colname)
    colname = re.sub(r"(SALUD_HOMBRE_)", "", colname)
    colname = re.sub(r"(SALUD_MUJER_)", "", colname)
    colname = re.sub(r"(\d)", "", colname)
    return colname

def to_atc(value):
    return re.sub(r"([,;].*$)", "", value)

def to_name(value):
    return re.sub(r"(^.*[,;])", "", value)

coded_meds.variable = coded_meds.variable.apply(lambda x: to_illness(x))
coded_meds['CODI_ATC'] = coded_meds.value.apply(lambda x: to_atc(x))
coded_meds['NOMBRE'] = coded_meds.value.apply(lambda x: to_name(x))

coded_meds[coded_meds.variable == 'HTA'].head()

,entity_id,variable,value,CODI_ATC,NOMBRE
295874,=E00251415248321,HTA,C09AA;LISINOPRIL,C09AA,LISINOPRIL
295891,=E00251415423621,HTA,C09AA;ENALAPRIL,C09AA,ENALAPRIL
295902,=E00251429233421,HTA,C09AA;ENALAPRIL,C09AA,ENALAPRIL
295906,=E00251424001121,HTA,C09CA;DIOVAN,C09CA,DIOVAN
295922,=E00251415553021,HTA,C09CA;LOSARTAN,C09CA,LOSARTAN


## Load ATC codes

In [7]:
atc = pd.read_csv('../inst/extdata/medications/Codi_ATC.csv', sep=';', encoding='latin1')
atc.head()

,CODI_ATC,NOMBRE
0,B01AC,AAS 100
1,N02BA,AAS 500
2,N02BA,AB FE CAMPS
3,J01EE,ABACTRIM
4,J01RA,ABACTRIM BALSAMICO


In [8]:
prostata = pd.read_csv('../inst/extdata/medications/Medicaments_Prostata.csv', sep=',')
prostata.head()

,CODI_ATC,NOMBRE
0,G03HA01,ANDROCUR
1,G04BC,ACALKA
2,G04BC,MICIFRONA
3,G04BC,ROWANEFRIN Sol. oleosa
4,G04BC,TUTUKON


In [9]:
hormones = pd.read_csv('../inst/extdata/medications/hormones.csv', sep=',')
hormones.head()

,CODI_ATC,NOMBRE
0,G02BA03,MIRENA
1,G02BB,CREMA CONTRACEPTIVA LANZAS
2,G02BB,NACHA ESPUMA
3,G02BB,NACHAàvulo
4,G02BB01,CIRCLET


## Select hand-written medication columns

In [10]:
others = list(all.columns[all.columns.str.contains('_MEDICACION_.*_OTROS')])
columns = others.copy()
columns.append('entity_id')

df = all.loc[:, all.columns.isin(columns)]

In [11]:
a = pd.melt(df, id_vars = ['entity_id'], value_vars = others)
a = a[~a.value.isnull()]
a.variable = a.variable.apply(lambda x: to_illness(x))
a[a.variable == 'HTA'].head()

,entity_id,variable,value
296001,=E00251432344121,HTA,Amlodipino 5
296021,=E00251415505421,HTA,euprenil
296050,=E00251429566221,HTA,enanopril
296053,=E00251417989621,HTA,ENNALAPRIL
296202,=E00251511133021,HTA,COVALS


In [12]:
import string

def clean(val):
    val = val.upper()
    val = val.translate(str.maketrans({key: None for key in string.punctuation}))
    val = re.sub(r"(\d+)", "", val)
    val = re.sub(r"( MG$)", "", val)
    val = re.sub(r"( G$)", "", val)
    val = val.strip()
    return val

def compare(x, atc, threshold):
    
    medications = list(map(lambda x:x.lower(), list(atc.NOMBRE)))
    ls = difflib.get_close_matches(x.lower(), medications, 1, threshold)

    ls = ''.join(ls)

    field_codi_name = "{}_{}".format("CODI_ATC", threshold)
    field_name_name = "{}_{}".format("NOMBRE", threshold)

    if ls in medications:
        return pd.Series({field_codi_name: atc.loc[medications.index(ls), 'CODI_ATC'],
                          field_name_name: atc.loc[medications.index(ls), 'NOMBRE']})
    return pd.Series({field_codi_name: None, field_name_name: None})

## Explore similarity thresholds

In [13]:
medications = pd.concat([atc, prostata, hormones], ignore_index=True)
b = a.copy()
b['clean'] = b['value'].apply(lambda x: clean(x))

In [14]:
c = b['clean'].apply(lambda x: compare(x, atc, 1))
b = pd.concat([b, c], axis=1)

c = b['clean'].apply(lambda x: compare(x, atc, 0.9))
b = pd.concat([b, c], axis=1)

c = b['clean'].apply(lambda x: compare(x, atc, 0.8))
b = pd.concat([b, c], axis=1)

c = b['clean'].apply(lambda x: compare(x, atc, 0.7))
b = pd.concat([b, c], axis=1)

## Merge categorical and hand-written medications

In [15]:
hand_written_meds = b
hand_written_meds = hand_written_meds.rename(columns = {'CODI_ATC_0.9' : 'CODI_ATC'})
hand_written_meds = hand_written_meds.rename(columns = {'NOMBRE_0.9' : 'NOMBRE'})
hand_written_meds = hand_written_meds[~hand_written_meds.CODI_ATC.isnull()]

In [16]:
columns = [
    'variable',
    'value',
    'CODI_ATC',
    'NOMBRE'
]
hand_written_meds[hand_written_meds.variable == 'HTA'][hand_written_meds.CODI_ATC_1.isnull()].head()[columns]

/software/debian-8/general/virtenvs/jupyter/lib/python3.5/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


,variable,value,CODI_ATC,NOMBRE
296053,HTA,ENNALAPRIL,C09AA,ENALAPRIL
296202,HTA,COVALS,C09DA,CO VALS
296497,HTA,Lisonopril,C09AA,LISINOPRIL
297185,HTA,CO-DIOVAN,C09DA,CO DIOVAN
297325,HTA,CO-DIOVAN,C09DA,CO DIOVAN


## Write all medication

In [18]:
columns = ['entity_id', 'variable', 'CODI_ATC', 'NOMBRE']
merged_meds = pd.concat([coded_meds[columns], hand_written_meds[columns]])
merged_meds.columns = ['entity_id', 'CONDITION', 'ATC_CODE', 'DRUG_NAME']
merged_meds.to_csv('../output/medications/data.csv', index=False)

## Write errors

In [19]:
errors = b
errors.to_csv('../output/medications/errors.csv', index=False)